In [1]:
import glob
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import rocks

rocks.set_log_level("error")
import time as t


from astropy.time import Time
import requests
import io

import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import angular_separation

from scipy.signal import argrelextrema

import ssptools

from figure_mask import compute_mask, print_statistics, compute_mask_spin

In [2]:
import figure_setup as fs

# Get ZTF fit

In [3]:
# Local Configuration
data_fink = "../"
bft_file = os.path.join(data_fink, "data", "ssoBFT-latest.parquet")

In [4]:
# ZTF filters 1: g, 2: r
filters = {"1": "g", "2": "r"}

S_color = {"g": -0.3928, "r": 0.2913}
sun_color = {"g": -0.3044, "r": 0.1903}

color_C = -(sun_color["g"] - sun_color["r"])
color_S = -(S_color["g"] - S_color["r"])

fink_colors = ["#15284F", "#F5622E"]

V_minus_g = -0.2833
V_minus_r = 0.1777

In [5]:
data = pd.read_parquet(os.path.join(data_fink, "data", "ztf", "sso_ZTF.parquet"))

In [6]:
data["SHG1G2_dSpin"] = np.sqrt(
    (data["SHG1G2_dalpha0"] * np.cos(np.radians(data["SHG1G2_delta0"]))) ** 2
    + data["SHG1G2_ddelta0"] ** 2
)


# # Remove solutions above 90 deg of latitude
cond = data.SHG1G2_delta0 > 90
data.loc[cond, "SHG1G2_delta0"] = 90
print(f"above 90: {len(data[cond])} ")

cond = data.SHG1G2_delta0 < -90
data.loc[cond, "SHG1G2_delta0"] = -90
print(f"below 90: {len(data[cond])} ")

above 90: 0 
below 90: 0 


In [7]:
cols = [
    "sso_number",
    "sso_name",
    "sso_class",
    "orbital_elements.semi_major_axis.value",
    "orbital_elements.eccentricity.value",
    "orbital_elements.inclination.value",
    "orbital_elements.node_longitude.value",
    "orbital_elements.perihelion_argument.value",
    "orbital_elements.mean_anomaly.value",
    "orbital_elements.mean_motion.value",
    "family.family_number",
    "family.family_name",
    "proper_elements.proper_semi_major_axis.value",
    "proper_elements.proper_eccentricity.value",
    "proper_elements.proper_inclination.value",
    "proper_elements.proper_sine_inclination.value",
    "tisserand_parameters.Jupiter.value",
    "albedo.value",
    "absolute_magnitude.value",
    "diameter.value",
    "taxonomy.class",
    "taxonomy.complex",
    "taxonomy.waverange",
    "taxonomy.scheme",
    "taxonomy.technique",
    "colors.g-r.color.value",
    "colors.g-r.color.error.min",
    "colors.g-r.color.error.max",
    "colors.g-r.facility",
    "colors.g-r.observer",
    "colors.g-r.epoch",
    "colors.g-r.delta_time",
    "colors.g-r.id_filter_1",
    "colors.g-r.id_filter_2",
    "colors.g-r.phot_sys",
    "colors.g-r.technique",
    "colors.g-i.color.value",
    "colors.g-i.color.error.min",
    "colors.g-i.color.error.max",
    "colors.g-i.facility",
    "colors.g-i.observer",
    "colors.g-i.epoch",
    "colors.g-i.delta_time",
    "colors.g-i.id_filter_1",
    "colors.g-i.id_filter_2",
    "colors.g-i.phot_sys",
    "colors.g-i.technique",
    "colors.i-z.color.value",
    "colors.i-z.color.error.min",
    "colors.i-z.color.error.max",
    "colors.i-z.facility",
    "colors.i-z.observer",
    "colors.i-z.epoch",
    "colors.i-z.delta_time",
    "colors.i-z.id_filter_1",
    "colors.i-z.id_filter_2",
    "colors.i-z.phot_sys",
    "colors.i-z.technique",
    "spins.1.obliquity",
    "spins.1.RA0.value",
    "spins.1.DEC0.value",
    "spins.1.RA0.error.max",
    "spins.1.DEC0.error.max",
    "spins.1.long.value",
    "spins.1.lat.value",
    "spins.1.technique",
    "spins.2.obliquity",
    "spins.2.RA0.value",
    "spins.2.DEC0.value",
    "spins.2.RA0.error.max",
    "spins.2.DEC0.error.max",
    "spins.2.long.value",
    "spins.2.lat.value",
    "spins.2.technique",
    "spins.3.obliquity",
    "spins.3.RA0.value",
    "spins.3.DEC0.value",
    "spins.3.RA0.error.max",
    "spins.3.DEC0.error.max",
    "spins.3.long.value",
    "spins.3.lat.value",
    "spins.3.technique",
    "spins.4.obliquity",
    "spins.4.RA0.value",
    "spins.4.DEC0.value",
    "spins.4.RA0.error.max",
    "spins.4.DEC0.error.max",
    "spins.4.long.value",
    "spins.4.lat.value",
    "spins.4.technique",
]
bft = pd.read_parquet(bft_file, columns=cols)

In [8]:
data = data.merge(bft[cols], left_on="name", right_on="sso_name", how="left")

In [9]:
# data

In [10]:
mask_SHG1G2_union = compute_mask(data, model='SHG1G2', kind='union')
mask_HG1G2_union = compute_mask(data, model='HG1G2', kind='union')
mask_HG_union = compute_mask(data, model='HG', kind='union')

mask_SHG1G2_inter = compute_mask(data, model='SHG1G2', kind='inter')
mask_HG1G2_inter = compute_mask(data, model='HG1G2', kind='inter')
mask_HG_inter = compute_mask(data, model='HG', kind='inter')

maskFINK_inter = compute_mask(data, R_min=0.305, model='FINK', kind='inter')
maskFINK_union = compute_mask(data, R_min=0.305, model='FINK', kind='union')

for model in ['HG', 'HG1G2', 'SHG1G2', 'FINK']:
    print_statistics(data, model)
    print()

  All data       : 122675  (100.00%)
  Mask HG (g∩r) : 120023  ( 97.84%)
  Mask HG (gUr) : 120024  ( 97.84%)

  All data       : 122675  (100.00%)
  Mask HG1G2 (g∩r) :  47177  ( 38.46%)
  Mask HG1G2 (gUr) :  85363  ( 69.58%)

  All data       : 122675  (100.00%)
  Mask SHG1G2 (g∩r) :  64279  ( 52.40%)
  Mask SHG1G2 (gUr) :  97579  ( 79.54%)

  All data       : 122675  (100.00%)
  Mask FINK (g∩r) :  63092  ( 51.43%)
  Mask FINK (gUr) :  95593  ( 77.92%)



In [11]:
maskFINK = maskFINK_union

# General description of the sample

In [12]:
data.loc[maskFINK, "sso_class"].value_counts()

MB>Middle       24481
MB>Inner        23000
MB>Outer        19684
Phocaea           862
Hungaria          855
Trojan            720
Mars-Crosser      701
MB>Cybele         338
MB>Hilda          246
NEA>Apollo         82
NEA>Amor           69
NEA>Aten            8
Centaur             2
NEA>Atira           2
KBO>SDO             1
Name: sso_class, dtype: int64

In [13]:
As = data.loc[maskFINK, "orbital_elements.semi_major_axis.value"]
Es = data.loc[maskFINK, "orbital_elements.eccentricity.value"]
node = data.loc[maskFINK, "orbital_elements.inclination.value"]

fig, ax = plt.subplots(figsize=(12, 8))

ax.scatter(As, np.sin(np.radians(node)), marker=".", alpha=0.2, s=2)


ax.set_xlim(1.5, 5.5)
ax.set_ylim(0, 0.6)
ax.set_xlabel("Semi-major axis (au)")
ax.set_ylabel(r"Sine of inclination ($\sin$i)")

# fig.savefig(f'{data_fink}plots/types.png', facecolor='white', dpi=150)

Text(0, 0.5, 'Sine of inclination ($\\sin$i)')

# Spins!

In [14]:
data.columns[data.columns.str.contains("alpha0")]

Index(['SHG1G2_alpha0', 'SHG1G2_dalpha0', 'SHG1G2_alpha0_alt'], dtype='object')

In [15]:
# Create alternative spins solutions
xax = "SHG1G2_alpha0"
yax = "SHG1G2_delta0"

# Already done at Fink level
# data["SHG1G2_alpha0_alt"] = (data["SHG1G2_alpha0"] + 180) % 360
# data["SHG1G2_delta0_alt"] = -data["SHG1G2_delta0"]


data["SHG1G2_alpha0_rand"] = data["SHG1G2_alpha0"]
data["SHG1G2_delta0_rand"] = data["SHG1G2_delta0"]
cond = (data.index % 2) == 0
data.loc[cond, "SHG1G2_alpha0_rand"] = data.loc[cond, "SHG1G2_alpha0_alt"]
data.loc[cond, "SHG1G2_delta0_rand"] = data.loc[cond, "SHG1G2_delta0_alt"]


# Convert to EC
coords = SkyCoord(
    ra=data["SHG1G2_alpha0"].values * u.deg,
    dec=data["SHG1G2_delta0"].values * u.deg,
    distance=200 * u.parsec,
    frame="hcrs",
)
data["lon"] = coords.heliocentricmeanecliptic.lon.value
data["lat"] = coords.heliocentricmeanecliptic.lat.value

coords = SkyCoord(
    ra=data["SHG1G2_alpha0_alt"].values * u.deg,
    dec=data["SHG1G2_delta0_alt"].values * u.deg,
    distance=200 * u.parsec,
    frame="hcrs",
)
data["lon_alt"] = coords.heliocentricmeanecliptic.lon.value
data["lat_alt"] = coords.heliocentricmeanecliptic.lat.value


coords = SkyCoord(
    ra=data["SHG1G2_alpha0_rand"].values * u.deg,
    dec=data["SHG1G2_delta0_rand"].values * u.deg,
    distance=200 * u.parsec,
    frame="hcrs",
)
data["lon_rand"] = coords.heliocentricmeanecliptic.lon.value
data["lat_rand"] = coords.heliocentricmeanecliptic.lat.value

In [16]:
# Obliquity of the spin
data["lon_orbit"] = data["orbital_elements.node_longitude.value"] - 90
data["lat_orbit"] = 90.0 - data["orbital_elements.inclination.value"]
data["obliquity"] = data[["lon", "lat", "lon_orbit", "lat_orbit"]].apply(
    lambda x: np.degrees(
        angular_separation(
            np.radians(x[0]), np.radians(x[1]), np.radians(x[2]), np.radians(x[3])
        )
    ),
    axis=1,
)
data["obliquity_alt"] = data[["lon_alt", "lat_alt", "lon_orbit", "lat_orbit"]].apply(
    lambda x: np.degrees(
        angular_separation(
            np.radians(x[0]), np.radians(x[1]), np.radians(x[2]), np.radians(x[3])
        )
    ),
    axis=1,
)

# Figure longitude vs inclination

In [17]:
Is = np.sin(np.deg2rad(data["orbital_elements.inclination.value"]))

# --------------------------------------------------------------------------------
fig, ax = plt.subplots(
    2,
    5,
    figsize=fs.figsize(1, aspect=3),
    # sharex=True,
    # sharey=True,
    gridspec_kw={
        "wspace": 0.02,
        "hspace": 0.02,
        "left": 0.1,
        "right": 0.98,
        "top": 0.98,
        "bottom": 0.13,
    },
)
m = maskFINK

# --------------------------------------------------------------------------------
# Distributions
step = 0.04
count = 0
for i in range(0, 2):
    for j in range(0, 5):
        mi = (Is[m] >= count * step) * (Is[m] < (count + 1) * step)
        ax[i, j].hist(data["lon_rand"][m][mi], bins=45, density=True, histtype="step")
        
        ax[i, j].set_ylim(0, 5.9e-3)
        ax[i, j].set_xlim(0, 360)

#         ax[i, j].text(
#             180,
#             5e-3,
#             r"{:.2f} $\leq \sin(i) <$ {:.2f}".format(count * step, (count + 1) * step),
#             ha="center",
#             fontsize="x-small",
#         )
#         ax[i, j].text(
#             180,
#             1e-3,
#             "{:d} ({:.2f}%)".format(len(data["lon_rand"][m][mi]),
#                 len(data["lon_rand"][m][mi]) / len(data["lon_rand"][m]) * 100
#             ),
#             ha="center",
#             fontsize="x-small",
#         )

        if i==0:
            yTit, yNum, yFrac = 0.5e-3, 5e-3, 5e-3
        else:
            yTit, yNum, yFrac = 5e-3, 0.5e-3, 0.5e-3
        
        ax[i, j].text(
            180,
            yTit,
            r"{:.2f} $\leq \sin(i) <$ {:.2f}".format(count * step, (count + 1) * step),
            ha="center",
            fontsize="x-small",
        )
        ax[i, j].text(
            20,
            yNum,
            "{:,d}".format(len(data["lon_rand"][m][mi])),
            ha="left",
            fontsize="x-small",
        )
        ax[i, j].text(
            340,
            yFrac,
            "({:.2f}%)".format(
                len(data["lon_rand"][m][mi]) / len(data["lon_rand"][m]) * 100
            ),
            ha="right",
            fontsize="x-small",
        )

        count += 1
        

# --------------------------------------------------------------------------------
# Axes
ax[1, 2].set_xlabel(r"Longitude of the spin ($^{\circ}$)")

for a in ax[:, 0]:
    a.set_ylabel(r"Density")

for a in ax[:, 1:].ravel():
    a.set_yticklabels(['', '', '', '', ''])
    
for a in ax[0:, 0:].ravel():
    a.set_xticklabels(['', '', '', '', ''])

for a in ax.ravel():
    a.set_xticks([0, 90, 180, 270, 360])
for a in ax[1, :-1]:
    a.set_xticklabels([0, 90, 180, 270, " "])
ax[1, -1].set_xticklabels([0, 90, 180, 270, 360])

# --------------------------------------------------------------------------------
fig.savefig(os.path.join(data_fink, "gfx", "article", "longitude_inclination.pgf"))
fig.savefig(
    os.path.join(data_fink, "gfx", "article", "longitude_inclination.png"),
    facecolor="white",
    dpi=180,
)

/tmp/ipykernel_690314/971050799.py:90: UserWarning: FixedFormatter should only be used together with FixedLocator
  a.set_yticklabels(['', '', '', '', ''])
/tmp/ipykernel_690314/971050799.py:93: UserWarning: FixedFormatter should only be used together with FixedLocator
  a.set_xticklabels(['', '', '', '', ''])
